In [1]:
%pip install pandas numpy scikit-learn joblib requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install torch torchvision torchaudio


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install pandas numpy scikit-learn xgboost joblib requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
from xgboost import XGBRegressor
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.ensemble import RandomForestRegressor
import joblib
import requests
import pickle
import json
from sklearn.model_selection import cross_val_score,KFold
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv('trainingdata_with_ps_50k.csv')
df = df.drop(['Weather_Severity'], axis=1)
df = pd.get_dummies(df, columns=['Weather_Condition'])#converts it into binary

In [7]:
x = df.drop(['Base_Priority', 'Business_Adjusted_Priority', 'Final_Priority', 'Normalized_Priority_Score'], axis=1)#features
y = df['Normalized_Priority_Score']#target

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Xgboost

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Xgboost

In [10]:

X_train_gs, X_val, y_train_gs, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)
xgb_reg = XGBRegressor(
    objective='reg:squarederror',
    eval_metric='rmse',
    max_depth=6,                  
    eta=0.05,                     
    subsample=0.9,                
    colsample_bytree=0.9,         
    reg_alpha=0.1,                
    reg_lambda=1.0,               
    n_estimators=500              
)

# Train with early stopping to monitor validation error
xgb_reg.fit(
    X_train_gs, y_train_gs,
    eval_set=[(X_val, y_val)],#early stopping
       
    verbose=False
)
results = xgb_reg.evals_result()


# Predict and evaluate
y_pred = xgb_reg.predict(X_val)

percentage_error = 100 * np.abs((y_val - y_pred) / y_val)
mean_percentage_error = np.nanmean(percentage_error)


print(f"Mean Percentage Error: {mean_percentage_error}")
joblib.dump(xgb_reg, 'xgb_model.joblib')
print("XGBoost model saved as 'xgb_model.joblib'.")

Mean Percentage Error: 2.4760042455158495
XGBoost model saved as 'xgb_model.joblib'.


In [11]:
cv_scores = cross_val_score(xgb_reg, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
print("Cross-Validation MSE:", -cv_scores.mean())

Cross-Validation MSE: 0.02641895926910324


In [12]:
loaded_xgb_model = joblib.load('xgb_model.joblib')
y_pred_xgb = loaded_xgb_model.predict(X_test_scaled)

In [13]:
xgb_reg.save_model('xgb_model.json')

Evaluate the xgboost model

In [14]:
rmse_xgb = root_mean_squared_error(y_test, y_pred_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

DNN 

In [15]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.relu = nn.LeakyReLU(negative_slope=0.01)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x
        

# Set up DNN model
input_size = X_train_scaled.shape[1]
hidden_size = 64
output_size = 1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dnn_model = DNNModel(input_size, output_size).to(device)

# Define loss and optimizer with weight decay for L2 regularization
criterion = nn.MSELoss()
optimizer = optim.Adam(dnn_model.parameters(), lr=0.001, weight_decay=0.001)

# Learning rate scheduler for dynamic adjustment
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

# Prepare data loaders
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1).to(device)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).unsqueeze(1).to(device)

# Training loop with early stopping
num_epochs = 100
patience = 5  # Early stopping patience
best_val_loss = np.inf
epochs_no_improve = 0

for epoch in range(num_epochs):
    dnn_model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = dnn_model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    # Calculate average training loss
    avg_train_loss = total_loss / len(train_loader)

    # Validation loss for early stopping
    dnn_model.eval()
    with torch.no_grad():
        val_outputs = dnn_model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor).item()
    
    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    # Adjust learning rate with scheduler
    scheduler.step(val_loss)

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(dnn_model.state_dict(), 'dnn_model_best.pth')  # Save best model
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping due to no improvement in validation loss.")
            break

# Load the best model for evaluation
dnn_model.load_state_dict(torch.load('dnn_model_best.pth'))
print("Training complete. Best model loaded.")

Epoch 1, Training Loss: 1.2240, Validation Loss: 0.0741
Epoch 2, Training Loss: 0.3169, Validation Loss: 0.0765
Epoch 3, Training Loss: 0.2909, Validation Loss: 0.0544
Epoch 4, Training Loss: 0.2675, Validation Loss: 0.0776
Epoch 5, Training Loss: 0.2519, Validation Loss: 0.0509
Epoch 6, Training Loss: 0.2452, Validation Loss: 0.0409
Epoch 7, Training Loss: 0.2496, Validation Loss: 0.0623
Epoch 8, Training Loss: 0.2327, Validation Loss: 0.0418
Epoch 9, Training Loss: 0.2307, Validation Loss: 0.0395
Epoch 10, Training Loss: 0.2223, Validation Loss: 0.0557
Epoch 11, Training Loss: 0.2116, Validation Loss: 0.0411
Epoch 12, Training Loss: 0.2154, Validation Loss: 0.0451
Epoch 13, Training Loss: 0.2113, Validation Loss: 0.0402
Epoch 14, Training Loss: 0.2063, Validation Loss: 0.0411
Early stopping due to no improvement in validation loss.
Training complete. Best model loaded.


C:\Users\lalitha priya.a\AppData\Local\Temp\ipykernel_8356\1782251258.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dnn_model.load_state_dict(torch.load('dnn_model_be

In [16]:


dnn_model.eval()

# Forward pass to get predictions on the validation set
with torch.no_grad():
    val_predictions = dnn_model(X_val_tensor).cpu().numpy()  # Convert to numpy for calculation
    y_val_true = y_val_tensor.cpu().numpy()
def mean_percentage_error(y_true, y_pred):
    percentage_errors = (y_true - y_pred) / y_true * 100  # Calculate percentage errors
    return np.mean(np.abs(percentage_errors))  # Return the mean of absolute percentage errors

mpe = mean_percentage_error(y_val_true, val_predictions)
print(f"Mean Percentage Error (MPE): {mpe:.2f}%")



Mean Percentage Error (MPE): 4.13%


In [17]:
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
dnn_model.eval()
with torch.no_grad():
    y_pred_dnn = dnn_model(X_test_tensor).cpu().numpy()

rmse_dnn = root_mean_squared_error(y_test, y_pred_dnn)
mae_dnn = mean_absolute_error(y_test, y_pred_dnn)
r2_dnn = r2_score(y_test, y_pred_dnn)

In [18]:
dnn_model_params = {k: v.cpu().tolist() for k, v in dnn_model.state_dict().items()}
with open('dnn_model.json', 'w') as json_file:
    json.dump(dnn_model_params, json_file)
print("DNN model saved as 'dnn_model.json'.")

DNN model saved as 'dnn_model.json'.


In [19]:
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

Meta Learner

In [20]:
xgb_pred = xgb_reg.fit(X_train, y_train).predict(X_test)

y_pred_combined = np.vstack((xgb_pred, y_pred_dnn.flatten())).T
meta_learner = RandomForestRegressor()
meta_learner.fit(y_pred_combined,y_test)
# Convert meta-learner parameters to dictionary before saving
meta_learner_params = meta_learner.get_params()




# Save parameters to JSON
with open('meta_learner.json', 'w') as json_file:
    json.dump(meta_learner_params, json_file)
print("Meta-learner parameters saved as 'meta_learner.json'.")

Meta-learner parameters saved as 'meta_learner.json'.


In [21]:
joblib.dump(meta_learner, 'meta_learner.pkl')
print("Meta-learner saved as 'meta_learner.pkl'.")

Meta-learner saved as 'meta_learner.pkl'.


In [22]:
loaded_meta_learner = joblib.load('meta_learner.pkl')
print("i")
y_pred_meta = loaded_meta_learner.predict(y_pred_combined)

percentage_error_meta = 100 * np.abs((y_test - y_pred_meta) / y_test)
mean_percentage_error_meta = np.mean(percentage_error_meta)
print(f"Mean Percentage Error (MPE) for Meta-Learner: {mean_percentage_error_meta:.2f}%")

rmse_meta = root_mean_squared_error(y_test, y_pred_meta)
mae_meta = mean_absolute_error(y_test, y_pred_meta)
r2_meta = r2_score(y_test, y_pred_meta)

i
Mean Percentage Error (MPE) for Meta-Learner: 1.03%


In [23]:
k = 5  
kf = KFold(n_splits=k, shuffle=True, random_state=42)


scoring = root_mean_squared_error(y_test, y_pred_meta)


scores = cross_val_score(meta_learner, y_pred_combined, y_test, cv=kf,)


mean_mae = np.mean(scores)
std_mae = np.std(scores)

print(f"{k}-Fold Cross-Validation Mean MAE: {mean_mae:.4f}")
print(f"{k}-Fold Cross-Validation MAE Standard Deviation: {std_mae:.4f}")

5-Fold Cross-Validation Mean MAE: 0.9920
5-Fold Cross-Validation MAE Standard Deviation: 0.0004


In [24]:
results = {
    'Model': ['XGBoost', 'DNN', 'Meta-Learner'],
    'RMSE': [rmse_xgb, rmse_dnn, rmse_meta],
    'MAE': [mae_xgb, mae_dnn, mae_meta],
    'R2 Score': [r2_xgb, r2_dnn, r2_meta]
}
results_df = pd.DataFrame(results)
results_df.to_csv('model_results.csv', index=False)

print("Model evaluation completed. Results saved to 'model_results.csv'.")

Model evaluation completed. Results saved to 'model_results.csv'.
